In [1]:
import numpy as np
import pandas as pd
import os
import sys
CURRENT_DIR = os.path.dirname(os.getcwd())
sys.path.append(os.path.dirname(CURRENT_DIR+'/func'))
from func.photonic_func import Photonic

In [2]:
import numpy as np
from plotly.offline import init_notebook_mode, iplot 
import plotly.graph_objs as go
init_notebook_mode(connected=True)  # for Jupyter Lab notebook

photonic = Photonic(config='fake_tof_night_1550')
dist_vec = np.array([0.2,0.5,1,2,5,10])
trace0 = go.Scatter(x=dist_vec,
                    y=1000 * photonic.wallFlux(dist_vec=dist_vec), mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                    name='wallFlux [W/m**2]')
trace01 = go.Scatter(x=dist_vec,
                    y=1000 * photonic.wallFlux(dist_vec=dist_vec, light_type='solar'), mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                    name='Solar wallFlux [W/m**2]')
trace1 = go.Scatter(x=dist_vec,
                    y=1000 * photonic.siliconFlux2(dist_vec=dist_vec), mode='lines+markers',
                    name='siliconFlux [W/m**2]')
trace2 = go.Scatter(x=dist_vec,
                    y=photonic.photoelectron2(dist_vec=dist_vec), mode='lines+markers',
                    name='photoelectrons / burst')
trace3 = go.Scatter(x=[1],
                    y=[0.01], mode='text', textposition='top right',
                    name='text', text=['Photonic simulation of light created by a light source attached to a camera<br>'
                    +'1. Flux is calculated on a wall at a certain distance assuming CW lighting mode<br>'
                    +'2. Flux on the focal plane of the silicon sensor as imaged from the wall thru the lens<br>'
                    +'3. Photoelectrons per a single burst collect in the photodiode of the pixel'])

data = [trace0, trace01, trace1, trace2, trace3]

layout = dict(title='Photonic simulation - Flux on wall/sensor and PE count',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='Flux [mW/m**2], PE [e-]',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

In [3]:
# test_cfg = ['fake_tof_night_850','fake_tof_day_850',
#             'fake_tof_night_940','fake_tof_day_940',
#             'fake_tof_night_1375','fake_tof_day_1375',
#             'fake_tof_night_1550','fake_tof_day_1550']
data = [0,1,2,3,4,5,6,7, 8, 9, 10, 11, 12, 13, 14, 15]
test_cfg = ['fake_tof_day_940','fake_tof_day_1375']
data = [0,1,2,3]
for i, cfg in enumerate(test_cfg):
    photonic = Photonic(config=cfg)
    dist_vec = np.array([0.5,1,2,5,10])
    trace0 = go.Scatter(x=dist_vec,
                        y=1000 * photonic.wallFlux(dist_vec=dist_vec),   # Select 'lines', 'markers' or 'lines+markers'
                        mode='lines+markers', name=cfg[9:]+' IR')
    trace1 = go.Scatter(x=dist_vec,
                        y=1000 * photonic.wallFlux(dist_vec=dist_vec, light_type='solar'), mode='lines+markers',  # Select 'lines', 'markers' or 'lines+markers'
                        name=(cfg[9:]+' Solar'))
    data[2*i] = trace0
    data[2*i+1] = trace1

layout = dict(title='Photonic simulation - Flux on wall - Fake ToF - Constrained to Same Illumination Power',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='Flux [mW/m**2]',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

In [4]:
test_cfg = ['fake_tof_night_850','fake_tof_day_850',
            'fake_tof_night_940','fake_tof_day_940',
            'fake_tof_night_1375','fake_tof_day_1375',
            'fake_tof_night_1550','fake_tof_day_1550']
data = [0,1,2,3,4,5,6,7]
data2 = [0,1,2,3,4,5,6,7]
# test_cfg = ['fake_tof_day_940','fake_tof_day_1375']
# data = [0,1]
for i, cfg in enumerate(test_cfg):
    photonic = Photonic(config=cfg)
    dist_vec = np.array([0.5,0.7,1,1.5,2,3,4,5,7,10])
    signal, noise, SNR = photonic.signal_to_noise_ratio(dist_vec=dist_vec)
    trace0 = go.Scatter(x=dist_vec, y=sum(noise.values()),   # Select 'lines', 'markers' or 'lines+markers'
                        mode='lines+markers', name=cfg[9:])
    data[i] = trace0
    trace1 = go.Scatter(x=dist_vec, y=SNR,   # Select 'lines', 'markers' or 'lines+markers'
                        mode='lines+markers', name=cfg[9:])
    data2[i] = trace1

layout = dict(title='Photonic simulation - Summed Noise',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='Noise [e-]',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

In [5]:
layout = dict(title='Photonic simulation - SNR',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='SNR',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data2, layout=layout))

In [20]:
# test_cfg = ['fake_tof_night_850','fake_tof_day_850',
#             'fake_tof_night_940','fake_tof_day_940',
#             'fake_tof_night_1375','fake_tof_day_1375',
#             'fake_tof_night_1550','fake_tof_day_1550']
# data = [0,1,2,3,4,5,6,7]
cfg = 'fake_tof_day_1375'
# data = [0]
# for i, cfg in enumerate(test_cfg):
photonic = Photonic(config=cfg)
dist_vec = np.array([0.5,0.7,1,1.5,2,3,4,5,7,10])
signal, noise, SNR = photonic.signal_to_noise_ratio(dist_vec=dist_vec)
sz = len(noise['noise_photon_shot'])

trace0 = go.Scatter(x=dist_vec, y=noise['noise_photon_shot'],   # Select 'lines', 'markers' or 'lines+markers'
                    mode='lines+markers', name='photon shot '+cfg[9:])
trace1 = go.Scatter(x=dist_vec, y=sz*[noise['dark_noise']],   # Select 'lines', 'markers' or 'lines+markers'
                    mode='lines+markers', name='dark '+cfg[9:])
trace2 = go.Scatter(x=dist_vec, y=sz*[noise['kTC_noise']],   # Select 'lines', 'markers' or 'lines+markers'
                    mode='lines+markers', name='kTC '+cfg[9:])
data = [trace0, trace1, trace2]

layout = dict(title='Photonic simulation - Major Noise Sources',
              xaxis=dict(title='Wall Distance [m]', type='log'),  # Select 'log' or 'linear'
              yaxis=dict(title='Noise [e-]',type='log'),  # Select 'log' or 'linear'
              template='plotly_dark',
              barmode='group',
              hovermode='x')

iplot(dict(data=data, layout=layout))

In [7]:
cfg = 'fake_tof_day_1375'
photonic = Photonic(config=cfg)
dist_vec = np.array([0.5,0.7,1,1.5,2,3,4,5,7,10])
signal, noise, SNR = photonic.signal_to_noise_ratio(dist_vec=dist_vec)


